<a href="https://colab.research.google.com/github/yuyehara/IS-GEO2022_Angeve/blob/main/clippingRaster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install geopandas
!pip install earthpy
!pip install rasterio
!pip install rioxarray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 5.0 MB/s 
     |████████████████████████████████| 16.7 MB 48.4 MB/s 
     |████████████████████████████████| 6.3 MB 42.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 5.2 MB/s 
     |████████████████████████████████| 19.3 MB 1.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 47 kB 2.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for rioxarray: filename=rioxarray-0.9.1-py3-none-any.whl size=54611 sha256=2ececfee6e21a2cb2f0d4aef61578a7df6c9bb5f12

In [3]:
import os
import rioxarray as rxr
import rasterio
import geopandas as gpd


In [4]:
# Original code ref.: https://www.earthdatascience.org/courses/use-data-open-source-python/intro-raster-data-python/raster-data-processing/crop-raster-data-with-shapefile-in-python/
# import libraries
# import os
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np
# from shapely.geometry import mapping
# import rasterio
# import geopandas as gpd0
0# import earthpy as et
# import earthpy.plot as ep

0

In [5]:
# Prettier plotting with seaborn
# sns.set(font_scale=1.5)

NameError: ignored

In [11]:
# get vector using geopanda
palm_0339_poly = gpd.read_file("/content/drive/MyDrive/IS-GEO-22/palm_0339_2.shp")

# get a raster
palm_0339_fn = "/content/drive/MyDrive/IS-GEO-22/img_0339.tif"
palm_0339_ras = rasterio.open("/content/drive/MyDrive/IS-GEO-22/img_0339.tif")

In [16]:
import fiona
from pyproj import Proj, transform
polyFile = fiona.open('/content/drive/MyDrive/IS-GEO-22/palm_0339_2.shp')
crs_poly = Proj(polyFile.crs)
print(crs_poly)

palm_0339_poly.Id = range(palm_0339_poly.shape[0])

proj=utm zone=4 datum=WGS84 units=m no_defs ellps=WGS84 towgs84=0,0,0


In [8]:
# check the coodinate reference system (crs)
# project vector on the raster layer because the all raster layers have the same spatial reference
# the traingin data set are derived from the raster layers

polyCrs = palm_0339_poly.crs
rasCrs = palm_0339_ras.crs

palm_0339_poly.to_crs(rasCrs, inplace = True)
print(palm_0339_poly.crs)

print("palm_0339_poly =", polyCrs)
print("palm_0339_ras =", rasCrs)

PROJCS["WGS_1984_UTM_Zone_4N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-159],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32604"]]
palm_0339_poly = PROJCRS["WGS 84 / UTM zone 4N",BASEGEOGCRS["WGS 84",DATUM["World Geodetic System 1984",ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]],ID["EPSG",6326]],PRIMEM["Greenwich",0,ANGLEUNIT["Degree",0.0174532925199433]]],CONVERSION["UTM zone 4N",METHOD["Transverse Mercator",ID["EPSG",9807]],PARAMETER["Latitude of natural origin",0,ANGLEUNIT["Degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["

In [9]:
out_meta = palm_0339_ras.meta

In [10]:
def rasterio_write(palm_0339_ras, palm_0339_ras_break, shape):
  with rasterio.open(palm_0339_ras) as src:
    out_image, out_transform = rasterio.mask.mask(src, shape, crop=True)
    out_meta = src.meta

  out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
  
  return out_image, out_meta # new data


  # with rasterio.open(palm_0339_ras_break, "w", **out_meta) as dest:
  #   dest.write(out_image)


In [ ]:
# create a new file by opening up a raster file, then crop the raster by each polygon and name each one
palm_0339_poly[['Id','geometry']].apply(lambda polygon: rasterio_write(palm_0339_fn, palm_0339_fn + str(polygon[0]) + ".tif", polygon[1])
